<a href="https://colab.research.google.com/github/RtjShreyD/Eng-Mandarin/blob/master/train_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

In [9]:
!pip install torchvision

  Using cached https://files.pythonhosted.org/packages/88/95/90e8c4c31cfc67248bf944ba42029295b77159982f532c5689bcfe4e9108/torch-1.3.1-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: torch 0.3.0.post4
    Uninstalling torch-0.3.0.post4:
      Successfully uninstalled torch-0.3.0.post4


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
!pip install allennlp

     |████████████████████████████████| 7.6MB 8.9MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 256kB 72.8MB/s 
     |████████████████████████████████| 245kB 48.3MB/s 
     |████████████████████████████████| 194kB 62.2MB/s 
     |████████████████████████████████| 163kB 59.9MB/s 
     |████████████████████████████████| 133kB 63.8MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 1.0MB 46.8MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-cp36-none-any.whl size=42709 sha256=095be23b906984765fa4e04677e9f99ba028c71c5d23b4e4fab7da0e3a9bc5b8
  Stored in directory: /root/.cache/pip/wheels/b7/8d/e7/a0e74217da5caeb3c1c7689639b6d28ddbf9985b840bc96a9a
  Created wheel for numpydoc: filename=numpydoc-0.9.2-cp36-none-any.whl size=31894 sha256=777f71d316f6db8d8402cc68526e9c58593bfcc44a6b1501fee87585b2efe5c1
  Stored in directory: /root/.cache/pip/wheels/96/f3/52/25c8e1f40637661d27fe

In [0]:
import itertools

import torch
import torch.optim as optim
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.nn.activations import Activation
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.modules.attention import LinearAttention, BilinearAttention, DotProductAttention
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper, StackedSelfAttentionEncoder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.predictors import SimpleSeq2SeqPredictor
from allennlp.training.trainer import Trainer

EN_EMBEDDING_DIM = 256
ZH_EMBEDDING_DIM = 256
HIDDEN_DIM = 256

In [13]:
reader = Seq2SeqDatasetReader(
        source_tokenizer=WordTokenizer(),
        target_tokenizer=CharacterTokenizer(),
        source_token_indexers={'tokens': SingleIdTokenIndexer()},
        target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})
    
train_dataset = reader.read('/content/drive/My Drive/Eng_Mandarin/data/tatoeba.eng_cmn.train.tsv')
validation_dataset = reader.read('/content/drive/My Drive/Eng_Mandarin/data/tatoeba.eng_cmn.dev.tsv')

vocab = Vocabulary.from_instances(train_dataset + validation_dataset, 
                                  min_count={'tokens': 3, 'target_tokens': 3})

en_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'), 
                         embedding_dim=EN_EMBEDDING_DIM)

37400it [00:14, 2607.59it/s]
4676it [00:01, 2699.48it/s]
100%|██████████| 42076/42076 [00:00<00:00, 72356.80it/s]


In [0]:
encoder = StackedSelfAttentionEncoder(
        input_dim=EN_EMBEDDING_DIM, 
        hidden_dim=HIDDEN_DIM, 
        projection_dim=128, 
        feedforward_hidden_dim=128, 
        num_layers=1, 
        num_attention_heads=8)

source_embedder = BasicTextFieldEmbedder({"tokens": en_embedding})

In [15]:
attention = DotProductAttention()

max_decoding_steps = 20 

model = SimpleSeq2Seq(vocab, source_embedder, encoder, max_decoding_steps,
                      target_embedding_dim=ZH_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      attention=attention,
                      beam_size=8,
                      use_bleu=True)

if torch.cuda.is_available():
  CUDA_DEVICE = 0
  model = model.cuda(CUDA_DEVICE)
  print("Model deployed on GPU")
else:
  CUDA_DEVICE = -1
  print("Model deployed on CPU")

optimizer = optim.Adam(model.parameters())

iterator = BucketIterator(batch_size=32, sorting_keys=[("source_tokens", "num_tokens")])

iterator.index_with(vocab)

Model deployed on GPU


In [0]:
trainer = Trainer(model=model, 
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  num_epochs=50,
                  patience=1,
                  cuda_device=CUDA_DEVICE)
                     
trainer.train()
predictor = SimpleSeq2SeqPredictor(model, reader)

for instance in itertools.islice(validation_dataset, 10):
  print('SOURCE:', instance.fields['source_tokens'].tokens)
  print('GOLD:', instance.fields['target_tokens'].tokens)
  print('PRED:', predictor.predict_instance(instance)['predicted_tokens'])
  print('////////////////////////////////////////////\n')

with open("/content/drive/My Drive/Eng_Mandarin/data/model.th", 'wb') as f:
  torch.save(model.state_dict(), f)
vocab.save_to_files("/content/drive/My Drive/Eng_Mandarin/data/vocabulary")

In [18]:
vocab2 = Vocabulary.from_files("/content/drive/My Drive/Eng_Mandarin/data/vocabulary")
model2 = SimpleSeq2Seq(vocab2, source_embedder, encoder, max_decoding_steps,
                      target_embedding_dim=ZH_EMBEDDING_DIM,
                      target_namespace='target_tokens',
                      attention=attention,
                      beam_size=8,
                      use_bleu=True)

with open("/content/drive/My Drive/Eng_Mandarin/data/model.th", 'rb') as f:
  model2.load_state_dict(torch.load(f))

if CUDA_DEVICE > -1:
    model2.cuda(CUDA_DEVICE)

predictor2 = SimpleSeq2SeqPredictor(model2, reader)

for instance in itertools.islice(validation_dataset, 10):
  print('SOURCE:', instance.fields['source_tokens'].tokens)
  print('GOLD:', instance.fields['target_tokens'].tokens)
  print('PRED:', predictor2.predict_instance(instance)['predicted_tokens'])
  print('////////////////////////////////////////////\n')

SOURCE: [@start@, I, have, to, go, to, sleep, ., @end@]
GOLD: [@start@, 我, 该, 去, 睡, 觉, 了, 。, @end@]
PRED: ['我', '必', '睡', '睡', '觉', '。', '。']
////////////////////////////////////////////

SOURCE: [@start@, I, just, do, n't, know, what, to, say, ., @end@]
GOLD: [@start@, 我, 就, 是, 不, 知, 道, 說, 些, 什, 麼, 。, @end@]
PRED: ['我', '不', '知', '不', '知', '道', '的', '的', '什', '麼', '。']
////////////////////////////////////////////

SOURCE: [@start@, I, may, give, up, soon, and, just, nap, instead, ., @end@]
GOLD: [@start@, 也, 许, 我, 会, 马, 上, 放, 弃, 然, 后, 去, 睡, 一, 觉, 。, @end@]
PRED: ['我', '许', '我', '很', '开', '上', '去', '松', '。', '后', '。', '。', '觉', '步', '。']
////////////////////////////////////////////

SOURCE: [@start@, I, 'm, going, to, go, ., @end@]
GOLD: [@start@, 我, 要, 走, 了, 。, @end@]
PRED: ['我', '要', '去', '了', '。']
////////////////////////////////////////////

SOURCE: [@start@, That, 's, MY, line, !, @end@]
GOLD: [@start@, 那, 是, 我, 的, 台, 词, ！, @end@]
PRED: ['那', '是', '一', '的', '一', '@@UNKNOWN@@', '！'